In [17]:
import pathlib
import pandas as pd
import numpy as np

rootdir = pathlib.Path('.').resolve(strict=True)
datadir = rootdir.parents [1] / 'ace2-rbd-point-mutation-benchmark/files/input_files' # githuba koyup analizleri yaptığımız dosyalar
outputdir = rootdir.parents [1] / 'ace2-rbd-point-mutation-benchmark/files/output_files'
newfiles = rootdir.parents [1] / 'ace2-rbd-point-mutation-benchmark/run_scripts_errors/files' # yeni ürettiğimiz scorelar

# import dataset
ACE2 = pd.read_csv(datadir / 'ACE2_Experimental_dataset.csv', delimiter=',')
ACE2.columns=['#case_id','exp_binding']
#create protein column to represent which case belong which protein 
ACE2['protein'] = "ACE2"

RBD = pd.read_csv(datadir / 'RBD_Experimental_dataset.csv', delimiter=',')
RBD = RBD[['#case_id', 'RBD_bind_avg']]
RBD.columns=['#case_id','exp_binding']
RBD['protein'] = "RBD"

# concatanate ACE2 and RBD dataset to build experimental dataset of our study
Experimental_dataset = pd.concat([ACE2,RBD])
# setting case id as an index is necessary for joining datasets in an accurate order to the next level
Experimental_dataset=Experimental_dataset.set_index('#case_id')


In [2]:
FoldX_originalpdb = pd.read_csv(newfiles / 'FoldX_originalpdb.csv', delimiter=',')
FoldX_originalpdb
# calculate ∆∆G
FoldX_originalpdb['foldx-ddg'] = FoldX_originalpdb['mutant_originalpdb'] - FoldX_originalpdb['wt_originalpdb']

# remove foldx-score-mut and foldx-score-wt column
FoldX_scores = FoldX_originalpdb.drop(labels=['mutant_originalpdb', 'wt_originalpdb'], axis=1)
FoldX_scores=FoldX_scores.set_index('#case_id')

predictors = [Experimental_dataset, FoldX_scores]
contol_dataset = pd.concat(predictors, axis=1,join="inner")


In [3]:
pd.set_option('mode.chained_assignment', None)
def prediction_status(df):
    columns=df.columns
    ddg_columns=columns[columns.str.contains('ddg')]

    for i in ddg_columns:
        enriched = df[df['exp_binding']>0]
        depleted = df[df['exp_binding']<0]

        condition_enr = [
                (enriched.loc[:, [f'{i}']] < 0),
                (enriched.loc[:, [f'{i}']] >= 0)]

        value_enr = ['success', 'failure']
        enriched.loc[:,[f'{i}-prediction']] = np.select(condition_enr, value_enr)

        condition_dep = [
                (depleted.loc[:, [f'{i}']] <= 0),
                (depleted.loc[:, [f'{i}']] > 0)]

        value_dep = ['failure', 'success']
        depleted.loc[:,[f'{i}-prediction']] = np.select(condition_dep, value_dep)
        df=pd.concat([enriched, depleted])
    return (df)


In [4]:
contol=prediction_status(contol_dataset)
contol.columns=['exp_binding','protein','originalpdb_foldx-ddg', 'originalpdb_foldx-ddg-prediction']


In [5]:
RBD_ACE2_benchmarking_dataset = pd.read_csv(outputdir / 'SARS-CoV-2-RBD_ACE2_benchmarking_dataset.csv', delimiter=',')
data=RBD_ACE2_benchmarking_dataset[['#case_id','foldx-ddg', 'foldx-ddg-prediction']]
data=data.set_index('#case_id')

main_dataset =pd.concat([contol,data],axis=1)
main_dataset = main_dataset.reset_index()


In [6]:
main_dataset_list = main_dataset.values.tolist()             
diff_cases = []
for i in main_dataset_list:
    a=i[4]
    b=i[6]
    if a != b:
        x=i[0]
        diff_cases.append(x)
print(diff_cases)   
len(diff_cases)


['A386L', 'D30T', 'M82R', 'N330H', 'Q325T', 'Q325Y', 'Q42C', 'Q42H', 'Q42K', 'Q42L', 'Q42R', 'Q42T', 'Q42V', 'R393K', 'T27H', 'T324Q', 'L455M', 'V503L', 'D30P', 'G354Q', 'H34K', 'K353R', 'L45K', 'Q24K', 'Y41M', 'R403W']


26

In [7]:
df = pd.DataFrame()
for i in diff_cases:
    x = main_dataset[main_dataset['index']==i]
    df = pd.concat([df, x], axis=0)
df

,index,exp_binding,protein,originalpdb_foldx-ddg,originalpdb_foldx-ddg-prediction,foldx-ddg,foldx-ddg-prediction
1,A386L,2.678300,ACE2,-0.32,success,0.12,failure
5,D30T,0.537893,ACE2,0.06,failure,-0.67,success
34,M82R,1.087940,ACE2,-0.01,success,0.46,failure
36,N330H,2.641170,ACE2,0.01,failure,-0.05,success
45,Q325T,0.382183,ACE2,0.01,failure,-0.01,success
46,Q325Y,0.365574,ACE2,0.01,failure,-0.97,success
47,Q42C,2.854290,ACE2,0.00,failure,-0.01,success
48,Q42H,1.396360,ACE2,-0.46,success,0.03,failure
50,Q42K,1.729240,ACE2,0.02,failure,-0.05,success
51,Q42L,2.742130,ACE2,-0.31,success,0.01,failure


## Performance

In [8]:
def performance_calculation(df):
    """
    This fuction calculates performance of predictors from input dataset by using <xx>-prediction columns. 
    These columns represent the binary prediction status of predictor -success or failure.
    This function counts success cases and calculates the performance of predictor.
    """
    columns=df.columns
    # get <xx>-prediction columns as a list
    prediction_columns=columns[columns.str.contains('prediction')]
    performances = []
    for i in prediction_columns:
        predictor_name = f'{i}'
        success_rate= [f'{predictor_name}', round(sum(df[f'{i}'].str.count("success")/len(df['index'])*100),2)]
        performances.append(success_rate)
    performances = pd.DataFrame(performances)
    
    return performances

In [9]:
# Total performances of predictors were calculated. 
# Additionally performance calculation according to mutation type(Enriched, Depleted) and protein type (ACE2, RBD) were builded.  

Total=performance_calculation(main_dataset)
Enriched=performance_calculation(main_dataset[main_dataset['exp_binding']>0])
Depleted=performance_calculation(main_dataset[main_dataset['exp_binding']<0])
ACE2=performance_calculation(main_dataset[main_dataset.protein =='ACE2'])
ACE2_Enriched=performance_calculation(main_dataset[(main_dataset.protein == 'ACE2') & (main_dataset['exp_binding'] >0)])
ACE2_Depleted=performance_calculation(main_dataset[(main_dataset.protein == 'ACE2') & (main_dataset['exp_binding'] <0)])
RBD=performance_calculation(main_dataset[main_dataset.protein =='RBD'])
RBD_Enriched=performance_calculation(main_dataset[(main_dataset.protein == 'RBD') & (main_dataset['exp_binding'] >0)])
RBD_Depleted=performance_calculation(main_dataset[(main_dataset.protein == 'RBD') & (main_dataset['exp_binding'] <0)])

main_performance_table = pd.concat([Total,Enriched[1], Depleted[1], ACE2[1], ACE2_Enriched[1], ACE2_Depleted[1], RBD[1], RBD_Enriched[1], RBD_Depleted[1]], axis=1)
main_performance_table.columns=['Predictors', 'Total', 'Enriched', 'Depleted', 'ACE2', 'ACE2 Enriched', 'ACE2 Depleted', 'RBD', 'RBD Enriched', 'RBD Depleted']
main_performance_table['Predictors'] = ['FoldX original-pdb', 'FoldX-github']
main_performance_table.set_index('Predictors')

,Total,Enriched,Depleted,ACE2,ACE2 Enriched,ACE2 Depleted,RBD,RBD Enriched,RBD Depleted
Predictors,,,,,,,,,
FoldX original-pdb,63.12,49.62,76.52,60.34,44.94,75.56,69.05,59.52,78.57
FoldX-github,63.88,51.15,76.52,62.01,47.19,76.67,67.86,59.52,76.19


## EvoEF1

In [25]:
EvoEF1_originalpdb = pd.read_csv(newfiles / 'EvoEF1_scores_originalpdb.csv', delimiter=',')

wt= EvoEF1_originalpdb[EvoEF1_originalpdb['#case_id'].str.contains("WT")]
wt_score = wt['evoef1-score'].values.tolist()

EvoEF1_originalpdb = EvoEF1_originalpdb.set_index("#case_id")
EvoEF1_originalpdb = EvoEF1_originalpdb.drop(labels=['WT'], axis=0)
EvoEF1_originalpdb = EvoEF1_originalpdb.reset_index()
EvoEF1_originalpdb = EvoEF1_originalpdb[['#case_id', 'evoef1-score']]
# calculate ddg
EvoEF1_originalpdb['evoef1-ddg'] = EvoEF1_originalpdb['evoef1-score'] - wt_score

# remove evoef1-score column
EvoEF1_originalpdb = EvoEF1_originalpdb.drop(labels=['evoef1-score'], axis=1)
EvoEF1_originalpdb=EvoEF1_originalpdb.set_index('#case_id')

predictors = [Experimental_dataset, EvoEF1_originalpdb]
contol_dataset = pd.concat(predictors, axis=1,join="inner")


In [24]:
contol=prediction_status(contol_dataset)
contol.columns=['exp_binding','protein','originalpdb_evoef-ddg', 'originalpdb_evoef-ddg-prediction']


In [32]:
data=RBD_ACE2_benchmarking_dataset[['#case_id','evoef1-ddg', 'evoef1-ddg-prediction']]
data=data.set_index('#case_id')

main_dataset =pd.concat([contol,data],axis=1)
main_dataset = main_dataset.reset_index()


In [29]:
main_dataset_list = main_dataset.values.tolist()             
diff_cases = []
for i in main_dataset_list:
    a=i[4]
    b=i[6]
    if a != b:
        x=i[0]
        diff_cases.append(x)
print(diff_cases)   
len(diff_cases)


['F28Y', 'H34S', 'K31W', 'Q493A', 'Q493G', 'Q493L', 'Q493V', 'Y453K', 'D38R', 'E37R', 'K31P', 'Y453D', 'Y453E', 'Y453P', 'Y489F']


15

In [30]:
df = pd.DataFrame()
for i in diff_cases:
    x = main_dataset[main_dataset['index']==i]
    df = pd.concat([df, x], axis=0)
df

,index,exp_binding,protein,originalpdb_evoef-ddg,originalpdb_evoef-ddg-prediction,evoef1-ddg,evoef1-ddg-prediction
13,F28Y,0.804817,ACE2,0.12,failure,-0.22,success
16,H34S,1.690540,ACE2,0.16,failure,-0.43,success
19,K31W,2.556800,ACE2,-1.62,success,0.10,failure
104,Q493A,0.130000,RBD,-0.21,success,0.75,failure
106,Q493G,0.010000,RBD,-0.19,success,0.77,failure
108,Q493L,0.050000,RBD,-0.85,success,0.06,failure
110,Q493V,0.050000,RBD,-0.41,success,0.54,failure
129,Y453K,0.080000,RBD,0.13,failure,-0.50,success
144,D38R,-2.367520,ACE2,-0.62,failure,0.28,success
154,E37R,-1.950860,ACE2,0.67,success,-0.60,failure


In [31]:
# Total performances of predictors were calculated. 
# Additionally performance calculation according to mutation type(Enriched, Depleted) and protein type (ACE2, RBD) were builded.  

Total=performance_calculation(main_dataset)
Enriched=performance_calculation(main_dataset[main_dataset['exp_binding']>0])
Depleted=performance_calculation(main_dataset[main_dataset['exp_binding']<0])
ACE2=performance_calculation(main_dataset[main_dataset.protein =='ACE2'])
ACE2_Enriched=performance_calculation(main_dataset[(main_dataset.protein == 'ACE2') & (main_dataset['exp_binding'] >0)])
ACE2_Depleted=performance_calculation(main_dataset[(main_dataset.protein == 'ACE2') & (main_dataset['exp_binding'] <0)])
RBD=performance_calculation(main_dataset[main_dataset.protein =='RBD'])
RBD_Enriched=performance_calculation(main_dataset[(main_dataset.protein == 'RBD') & (main_dataset['exp_binding'] >0)])
RBD_Depleted=performance_calculation(main_dataset[(main_dataset.protein == 'RBD') & (main_dataset['exp_binding'] <0)])

main_performance_table = pd.concat([Total,Enriched[1], Depleted[1], ACE2[1], ACE2_Enriched[1], ACE2_Depleted[1], RBD[1], RBD_Enriched[1], RBD_Depleted[1]], axis=1)
main_performance_table.columns=['Predictors', 'Total', 'Enriched', 'Depleted', 'ACE2', 'ACE2 Enriched', 'ACE2 Depleted', 'RBD', 'RBD Enriched', 'RBD Depleted']
main_performance_table['Predictors'] = ['EvoEF original-pdb', 'EvoEF github']
main_performance_table.set_index('Predictors')

,Total,Enriched,Depleted,ACE2,ACE2 Enriched,ACE2 Depleted,RBD,RBD Enriched,RBD Depleted
Predictors,,,,,,,,,
EvoEF original-pdb,58.56,50.38,66.67,56.42,47.19,65.56,63.10,57.14,69.05
EvoEF github,57.41,48.85,65.91,57.54,48.31,66.67,57.14,50.00,64.29
